In [6]:
import numpy as np
import seaborn as sns
import pandas as pd
import sys
import torch as t
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.optim as optim
import codecs 
sys.path.append('./model')
# from BiLSTM_CRF import BiLSTM_CRF
from resultCal import calculate
from sklearn.model_selection import train_test_split
import pickle
import pdb
device = t.device('cuda:4' if t.cuda.is_available() else 'cpu')

In [159]:
df = pd.read_csv('./res/text_crf.csv')
df.head(5)

FileNotFoundError: File b'./res/text_crf.csv' does not exist

In [191]:
df_test = pd.read_csv('./res/text_crf__no_e.csv')
df_base = pd.read_csv('./res/base_text.csv')
df_train = pd.read_csv('./data/train.csv')
test = ' '.join((df_test.n_crop+df_test.n_disease+df_test.n_medicine).values)
train = ' '.join((df_train.n_crop+df_train.n_disease+df_train.n_medicine).values)
base = ' '.join((df_base.n_crop+df_base.n_disease+df_base.n_medicine).values)

In [192]:
import re
test_list, train_list, base_list = [], [], []
for x in re.findall('\'(.*?)\'', test):
    test_list.append(x)
for x in re.findall('\'(.*?)\'', train):
    train_list.append(x)
for x in re.findall('\'(.*?)\'', base):
    base_list.append(x)

In [216]:
train_str = pd.read_csv('./data/train_clean.csv').clean_text.map(lambda x: [i.split('/')[0] for i in x.split(' ')]).values
train_set =  set()
for i in train_str:
    train_set.update("".join(i))
train_str_unique = list(train_set)

test_str = pd.read_csv('./data/test.csv').text.values
test_set =  set()
for i in test_str:
    test_set.update("".join(i))
test_str_unique = list(test_set)

In [218]:
for i in test_str_unique:
    if i not in train_str_unique: print(i)

蕾
蛀
塞
钻
飘
厚
扣
模
迹
臺
皱
园
蜂
瓶
糊
 
豆
孔
被
逐
熟
拂
?
捉
亚
穿
酵
:
噁
驳
按
淀
蓝
胎
他
装
拌
疤
来
热
么
铵
莎
抓
味
段
洁
衣
相
挂
尝
耗
零
例
茧
另
浏
薊
觉
狠
朗
/


In [185]:
different_list = []
for i in base_list:
    if i not in train_list:
        different_list.append(i)
different_list = list(set(different_list))
different_list


['烯酰嘧菌酯',
 '螺虫螨酯',
 '腐熟农家肥',
 '腐植酸叶面肥',
 '塞菌铜',
 '穿孔病',
 '多效唑',
 '溴螨酯',
 '细菌性果腐病',
 '海藻素',
 '高温障碍',
 '日烧病']

In [186]:
df = pd.read_csv('./data/train_clean_no_e.csv')
print(df[df.id=='689-3'].text.values)
print()
print(df[df.id=='689-3'].clean_text.values)

['根腐病/n_disease ，可以/n_new 在/p 发病初期/n_disease 用/v （/wkz 1）/wky 多菌灵/n_medicine +福美双/n_medicine +乙蒜素/n_medicine +生根剂/n_medicine （/wkz 2）/wky 恶霉灵/n_medicine +乙蒜素/n_medicine +生根剂/n_medicine （/wkz 3）/wky 枯草芽孢杆菌/n_disease +生根剂/n_medicine （/wkz 4）/wky 精/a 甲/n 咯菌腈/n_medicine +生根剂/n_medicine 灌根/n_new 淋/v 茎/n 处理/vn ']

['根/b_disease 腐/m_disease 病/m_disease ，/O 可/O 以/O 在/O 发/b_disease 病/m_disease 初/m_disease 期/m_disease 用/O （/O 1/O ）/O 多/b_medicine 菌/m_medicine 灵/m_medicine +/O 福/b_medicine 美/m_medicine 双/m_medicine +/O 乙/b_medicine 蒜/m_medicine 素/m_medicine +/O 生/b_medicine 根/m_medicine 剂/m_medicine （/O 2/O ）/O 恶/b_medicine 霉/m_medicine 灵/m_medicine +/O 乙/b_medicine 蒜/m_medicine 素/m_medicine +/O 生/b_medicine 根/m_medicine 剂/m_medicine （/O 3/O ）/O 枯/b_disease 草/m_disease 芽/m_disease 孢/m_disease 杆/m_disease 菌/m_disease +/O 生/b_medicine 根/m_medicine 剂/m_medicine （/O 4/O ）/O 精/O 甲/O 咯/b_medicine 菌/m_medicine 腈/m_medicine +/O 生/b_medicine 根/m_medicine 剂/m_medicine 灌/O 根/O 淋/O 茎/O 处/O 理/O']


In [187]:
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.optim as optim

torch.manual_seed(1)
START_TAG = "<START>"
STOP_TAG = "<STOP>"
def argmax(vec):
    # return the argmax as a python int
    _, idx = torch.max(vec, 1)
    return idx.item()


def prepare_sequence(seq, to_ix):
    idxs = [to_ix[w] for w in seq]
    return torch.tensor(idxs, dtype=torch.long)


# Compute log sum exp in a numerically stable way for the forward algorithm
def log_sum_exp(vec):
    max_score = vec[0, argmax(vec)]
    max_score_broadcast = max_score.view(1, -1).expand(1, vec.size()[1])
    return max_score + \
        torch.log(torch.sum(torch.exp(vec - max_score_broadcast)))

class BiLSTM_CRF(nn.Module):

    def __init__(self, vocab_size, tag_to_ix, embedding_dim, hidden_dim, device):
        super(BiLSTM_CRF, self).__init__()
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.vocab_size = vocab_size
        self.tag_to_ix = tag_to_ix
#         print(len(tag_to_ix))
        self.tagset_size = len(tag_to_ix)

        self.word_embeds = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim // 2,
                            num_layers=1, bidirectional=True)

        # Maps the output of the LSTM into tag space.
        self.hidden2tag = nn.Linear(hidden_dim, self.tagset_size)

        # Matrix of transition parameters.  Entry i,j is the score of
        # transitioning *to* i *from* j.
        self.transitions = nn.Parameter(
            torch.randn(self.tagset_size, self.tagset_size))

        # These two statements enforce the constraint that we never transfer
        # to the start tag and we never transfer from the stop tag
#         print(tag_to_ix[START_TAG])
        self.transitions.data[tag_to_ix[START_TAG], :] = -10000
        self.transitions.data[:, tag_to_ix[STOP_TAG]] = -10000
        self.device = device

    def init_hidden(self):
        weight = next(self.parameters())
        return (weight.new_zeros([2, 1, self.hidden_dim // 2]),
                weight.new_zeros([2, 1, self.hidden_dim // 2]))

    def _forward_alg(self, feats):
        weights = next(self.parameters())
        # Do the forward algorithm to compute the partition function
        init_alphas = weights.new_full((1, self.tagset_size), -10000.)
        # START_TAG has all of the score.
        init_alphas[0][self.tag_to_ix[START_TAG]] = 0.

        # Wrap in a variable so that we will get automatic backprop
        forward_var = init_alphas

        # Iterate through the sentence
        for feat in feats:
            alphas_t = []  # The forward tensors at this timestep
            for next_tag in range(self.tagset_size):
                # broadcast the emission score: it is the same regardless of
                # the previous tag
                emit_score = feat[next_tag].view(
                    1, -1).expand(1, self.tagset_size)
                # the ith entry of trans_score is the score of transitioning to
                # next_tag from i
                trans_score = self.transitions[next_tag].view(1, -1)
#                 print(trans_score.device, emit_score.device, forward_var.device)
                # The ith entry of next_tag_var is the value for the
                # edge (i -> next_tag) before we do log-sum-exp
                next_tag_var = forward_var + trans_score + emit_score
                # The forward variable for this tag is log-sum-exp of all the
                # scores.
                alphas_t.append(log_sum_exp(next_tag_var).view(1))
            forward_var = torch.cat(alphas_t).view(1, -1)
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        alpha = log_sum_exp(terminal_var)
        return alpha

    def _get_lstm_features(self, sentence):
        self.hidden = self.init_hidden()
        embeds = self.word_embeds(sentence).view(len(sentence), 1, -1)
        lstm_out, self.hidden = self.lstm(embeds, self.hidden)
        lstm_out = lstm_out.view(len(sentence), self.hidden_dim)
        lstm_feats = self.hidden2tag(lstm_out)
        return lstm_feats

    def _score_sentence(self, feats, tags):
        # Gives the score of a provided tag sequence
        score = torch.zeros(1).to(self.device)
        tags = torch.cat([torch.tensor([self.tag_to_ix[START_TAG]], dtype=torch.long).to(self.device), tags])
        for i, feat in enumerate(feats):
            score = score + self.transitions[tags[i + 1], tags[i]] + feat[tags[i + 1]]
        score = score + self.transitions[self.tag_to_ix[STOP_TAG], tags[-1]]
        return score

    def _viterbi_decode(self, feats):
        backpointers = []

        # Initialize the viterbi variables in log space
        init_vvars = torch.full((1, self.tagset_size), -10000.)
        init_vvars[0][self.tag_to_ix[START_TAG]] = 0

        # forward_var at step i holds the viterbi variables for step i-1
        forward_var = init_vvars
        for feat in feats:
            bptrs_t = []  # holds the backpointers for this step
            viterbivars_t = []  # holds the viterbi variables for this step

            for next_tag in range(self.tagset_size):
                # next_tag_var[i] holds the viterbi variable for tag i at the
                # previous step, plus the score of transitioning
                # from tag i to next_tag.
                # We don't include the emission scores here because the max
                # does not depend on them (we add them in below)
                next_tag_var = forward_var + self.transitions[next_tag]
                best_tag_id = argmax(next_tag_var)
                bptrs_t.append(best_tag_id)
                viterbivars_t.append(next_tag_var[0][best_tag_id].view(1))
            # Now add in the emission scores, and assign forward_var to the set
            # of viterbi variables we just computed
            forward_var = (torch.cat(viterbivars_t) + feat).view(1, -1)
            backpointers.append(bptrs_t)

        # Transition to STOP_TAG
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        best_tag_id = argmax(terminal_var)
        path_score = terminal_var[0][best_tag_id]

        # Follow the back pointers to decode the best path.
        best_path = [best_tag_id]
        for bptrs_t in reversed(backpointers):
            best_tag_id = bptrs_t[best_tag_id]
            best_path.append(best_tag_id)
        # Pop off the start tag (we dont want to return that to the caller)
        start = best_path.pop()
        assert start == self.tag_to_ix[START_TAG]  # Sanity check
        best_path.reverse()
        return path_score, best_path

    def neg_log_likelihood(self, sentence, tags):
        feats = self._get_lstm_features(sentence)
#         feats = feats.cpu()
#         print(feats.device)
        forward_score = self._forward_alg(feats)
        gold_score = self._score_sentence(feats, tags)
        return forward_score - gold_score

    def forward(self, sentence):  # dont confuse this with _forward_alg above.
        # Get the emission scores from the BiLSTM
#         self.hidden = self.init_hidden()
        lstm_feats = self._get_lstm_features(sentence)

        # Find the best path, given the features.
        score, tag_seq = self._viterbi_decode(lstm_feats)
        return score, tag_seq

In [188]:
with open('./data/maxlen256.pkl', 'rb') as inp:
    word2id = pickle.load(inp)
    id2word = pickle.load(inp)
    tag2id = pickle.load(inp)
    id2tag = pickle.load(inp)
    x = pickle.load(inp)
    y= pickle.load(inp)
print("train len:",len(x))

train len: 2790


In [189]:
START_TAG = "<START>"
STOP_TAG = "<STOP>"
EMBEDDING_DIM = 100
HIDDEN_DIM = 200
EPOCHS = 5
tag2id[START_TAG]=len(tag2id)
tag2id[STOP_TAG]=len(tag2id)
model = BiLSTM_CRF(len(word2id)+1, tag2id, EMBEDDING_DIM, HIDDEN_DIM, device).to(device)
optimizer = optim.SGD(model.parameters(), lr=0.005, weight_decay=1e-4)

In [190]:
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.1, shuffle=True, random_state=2020)
for epoch in range(EPOCHS):
    index=0
    for sentence, tags in zip(x_train,y_train):
        index+=1
        model.zero_grad()

        sentence=torch.tensor(sentence, dtype=torch.long).to(device)
        tags = torch.tensor([tag2id[t] for t in tags], dtype=torch.long).to(device)

        loss = model.neg_log_likelihood(sentence, tags)

        loss.backward()
        optimizer.step()
        if index%300==0:
            print("epoch",epoch,"index",index)
    entityres=[]
    entityall=[]
    for sentence, tags in zip(x_val,y_val):
        sentence=torch.tensor(sentence, dtype=torch.long).to(device)
        score,predict = model(sentence)
        entityres = calculate(sentence,predict,id2word,id2tag,entityres)
        entityall = calculate(sentence,tags,id2word,id2tag,entityall)
    jiaoji = [i for i in entityres if i in entityall]
    if len(jiaoji)!=0:
        zhun = float(len(jiaoji))/len(entityres)
        zhao = float(len(jiaoji))/len(entityall)
        print("test:")
        print("zhun:", zhun)
        print("zhao:", zhao)
        print("f:", (2*zhun*zhao)/(zhun+zhao))
    else:
        print("zhun:",0)
    
    path_name = "./pretrained_model/model_"+str(epoch)+".pt"
    print(path_name)
    torch.save(model, path_name)
    print("model has been saved")



KeyboardInterrupt: 

# PLOT

In [ ]:
import numpy as np
import seaborn as sns
import pandas as pd

In [ ]:
train_file = './data/train_clean.csv'
test_file = './data/test.csv'
df = pd.read_csv(train_file)
df_test = pd.read_csv(test_file)

In [ ]:
df.shape

In [ ]:
lens = df.clean_text.map(lambda x:len(x.split(' ')))
lens_test = df_test.text.map(len)
sns.distplot(lens, hist=False, label='train')
sns.distplot(lens_test, hist=False, label='test')

# Deal With datasets  (unused)

In [91]:
import pandas as pd
df = pd.read_csv('./data/train.csv')
def get_data(x):
    res = ''
    for words in x.split(' '):
        if '/' not in words:
            text_list = [i+'/O' for i in list(words)]
            res += " ".join(text_list)
            res += ' '
            continue            
        words = words.split('/')
        text, label = words[0], words[1]
        
        text_list = list(text)
        idx = len(text_list)-1
        while idx>=0 and text_list[idx] not in ['+', '，', '%', '·', '％']: idx -= 1
        #deal with samples which has %
        if idx>=0: 
            prev_str = " ".join([i+'/O' for i in text_list[:(idx+1)]])
            res += prev_str
            res += ' '  
        text_list = text_list[(idx+1):]
        if label == 'n_medicine': tag = 'medicine'
        elif label == 'n_disease': tag = 'disease'
        elif label == 'n_crop': tag = 'crop'
        else: tag='O'
        if tag != 'O':
            text_list[1:-1] = [i+'/m_'+tag for i in text_list[1:-1]]
            text_list[0] += '/b_'+tag
            text_list[-1] += '/e_'+tag
            res += " ".join(text_list)
        else:
            text_list = [i+'/O' for i in text_list]
            res += " ".join(text_list)
        res += ' '
    return res.strip()
df['clean_text'] = df.text.map(get_data)
df.to_csv('./data/train_clean.csv', index=False)

In [ ]:
def data2pkl():
    datas = list()
    labels = list()
    linedata=list()
    linelabel=list()
    tags = set()
    tags.add('')
    input_data = pd.read_csv('./data/train_clean.csv')['clean_text'].values
    for line in input_data:
        line = line.split()
        linedata=[]
        linelabel=[]
        numNotO=0
        for word in line:
            word = word.split('/')
            linedata.append(word[0])
            linelabel.append(word[1])
            tags.add(word[1])
            if word[1]!='O':
                numNotO+=1
        if numNotO!=0:
            datas.append(linedata)
            labels.append(linelabel)

    print(len(datas))
    print(len(labels))
    all_words = [i for data in datas for i in data]
    sr_allwords = pd.Series(all_words)
    sr_allwords = sr_allwords.value_counts()
    set_words = sr_allwords.index
    set_ids = range(1, len(set_words)+1)

    
    tags = [i for i in tags]
    tag_ids = range(len(tags))
    word2id = pd.Series(set_ids, index=set_words)
    id2word = pd.Series(set_words, index=set_ids)
    tag2id = pd.Series(tag_ids, index=tags)
    id2tag = pd.Series(tags, index=tag_ids)
    word2id["unknow"]=len(word2id)+1
    id2word[len(word2id)]="unknow"
    print(tag2id)
    max_len = 256
    def X_padding(words):
        ids = list(word2id[words])
        if len(ids) >= max_len:  
            return ids[:max_len]
        ids.extend([0]*(max_len-len(ids))) 
        return ids

    def y_padding(tags):
        ids = list(tag2id[tags])
        if len(ids) >= max_len: 
            return ids[:max_len]
        ids.extend([0]*(max_len-len(ids))) 
        return ids
    df_data = pd.DataFrame({'words': datas, 'tags': labels}, index=range(len(datas)))
    df_data['x'] = df_data['words'].apply(X_padding)
    df_data['y'] = df_data['tags'].apply(y_padding)
    x = np.asarray(list(df_data['x'].values))
    y = np.asarray(list(df_data['y'].values))
    
    import pickle
    import os
    with open('./data/maxlen256.pkl', 'wb') as outp:
        pickle.dump(word2id, outp)
        pickle.dump(id2word, outp)
        pickle.dump(tag2id, outp)
        pickle.dump(id2tag, outp)
        pickle.dump(x, outp)
        pickle.dump(y, outp)
    print('** Finished saving the data.')
data2pkl()

In [131]:
df.head(5)

,id,text,n_crop,n_disease,n_medicine,clean_text
0,1-1,疫病危害/n_disease ，建议选用/n_new 68·75%霜霉威/n_medicin...,[],['疫病危害'],"['霜霉威', '菌胺', '烯酰吗啉', '湿性粉剂']",疫/b_disease 病/m_disease 危/m_disease 害/e_diseas...
1,1-2,疫病/n_disease 炭疽病/n_disease 为害/v ，用/p 银法利/nrf 和...,[],"['疫病', '炭疽病']",['苯甲溴菌腈'],疫/b_disease 病/e_disease 炭/b_disease 疽/m_diseas...
2,1-3,炭疽病/n_disease 危害/v 使用/v 肟菌戊唑醇/n_medicine 或/c 苯...,[],['炭疽病'],"['肟菌戊唑醇', '苯甲丙环唑']",炭/b_disease 疽/m_disease 病/e_disease 危/O 害/O 使/...
3,2-1,高温灼伤/n_disease 或/c 药害/n_new 等/udeng 外力/n 损伤造成/...,[],['高温灼伤'],['碧护'],高/b_disease 温/m_disease 灼/m_disease 伤/e_diseas...
4,2-2,日/ng 灼/vg 斑/n 。/wj 叶面喷施/n_new 碧护/n_medicine 和/...,[],[],"['碧护', '螯合钙']",日/O 灼/O 斑/O 。/O 叶/O 面/O 喷/O 施/O 碧/b_medicine 护...


In [148]:
df = pd.read_csv('./data/train_clean.csv')
def train_clean_noend(x):
    res = []
    for i in x.split(' '):
        if '/' not in i: continue
        try:
            text, label = i.split('/')
        except:
            print(i)
        if label[0]=='e': label='m'+label[1:]
        res.append(text+'/'+label)
    return " ".join(res)
df.clean_text = df.clean_text.map(train_clean_noend)

In [150]:
df.to_csv('./data/train_clean_no_e.csv', index=False)

In [4]:
import pickle

In [6]:
train = pickle.load(open('./data/train.pkl', 'rb'))

In [8]:
train[0]

{'id': '1-1',
 'text': '生理性/n_new 缺钙/n_disease 。/wj 平衡/a 肥水/n 供应/vn ，花/v 前/f 硼/n 花/n 后/f 钙/n 喷雾/n_new 调理/v 。/wj',
 'text_list': ['生',
  '理',
  '性',
  '缺',
  '钙',
  '。',
  '平',
  '衡',
  '肥',
  '水',
  '供',
  '应',
  '，',
  '花',
  '前',
  '硼',
  '花',
  '后',
  '钙',
  '喷',
  '雾',
  '调',
  '理',
  '。'],
 'tag_list': ['O',
  'O',
  'O',
  'B-dise',
  'I-dise',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O'],
 'num_class_entity': 5}